In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings(action = 'ignore')

In [153]:
def dong_change(file_name):
    sample_df = pd.read_excel(file_name)
    sample_df.drop('STD_DAY',axis=1,inplace=True)

    change_df = pd.read_excel('행정구역분류.xlsx',header=1)

    seoul_df = change_df[change_df['시도']=='서울특별시']
    seoul_df = seoul_df[seoul_df['행정구역명'].str[-1] =='동']
    seoul_df = seoul_df[['시군구','행정구역명','법정동','법정동코드','행정기관코드','법정동 관할구역\n분할여부']]
    seoul_df.columns = ['GU','DONG','법정동','DONG_CODE','행정기관코드','분할여부']
    non_div = seoul_df[seoul_df['분할여부'].isnull()][['GU','DONG','DONG_CODE','행정기관코드']]

    remove_du = non_div.drop_duplicates(['행정기관코드'])

    tmp_sample_df = sample_df.drop(['GU','DONG'],axis=1)
    con_df1 = pd.merge(tmp_sample_df,non_div,on="DONG_CODE")

    non_div_result = con_df1.groupby('행정기관코드').sum()
    non_div_result = non_div_result.drop(['DONG_CODE'],axis=1)

    non_div_result = pd.merge(non_div_result, remove_du,right_on='행정기관코드',left_index=True)
    non_div_result.drop('DONG_CODE',inplace=True,axis=1)

    non_div_result.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)

    div_df = seoul_df[seoul_df['분할여부']=='분할연계'][['GU','DONG','DONG_CODE','행정기관코드']]

    div_dong_count = div_df.groupby('DONG_CODE').count()[['행정기관코드']]
    div_dong_count.columns = ['DONG_COUNT']

    con_df2 = pd.merge(sample_df, div_dong_count,left_on='DONG_CODE',right_index=True)
    con_df2.set_index('DONG_CODE',inplace=True)

    con_df2 = con_df2.iloc[:,3:]
    cal_df = con_df2.div(con_df2['DONG_COUNT'], axis=0) 
    div_result_df = pd.merge(div_df,cal_df,left_on='DONG_CODE',right_index=True)
    div_result_df = div_result_df.drop(['DONG_CODE','DONG_COUNT'],axis=1)
    div_result_df.rename(columns = {'행정기관코드':'DONG_CODE'},inplace=True)

    non_div_result = non_div_result[div_result_df.columns]
    result_df = pd.concat([non_div_result, div_result_df],axis=0)
    result_sum = result_df.groupby('DONG_CODE').sum()
    
    remove_result_du =result_df.drop_duplicates(['DONG_CODE'])[['GU','DONG','DONG_CODE']]
    result_df2 = pd.merge(result_sum, remove_result_du,right_on='DONG_CODE',left_index=True)

    return result_df2

In [154]:
def ps_remove_count():
    df = pd.read_excel('HOSPITAL_DM.xlsx',sheet_name='워크시트 익스포트')
    df = df[~df['HOSPITAL_NAME'].str.contains('성형')] # 성형외과 제거
    tmp_df = df[['HOSPITAL_CODE','HOSPITAL_NAME','HOSPITAL_TYPE','ADD_STR','DONG','DONG_CODE']]
    
    std = pd.read_excel('행정구역분류.xlsx',sheet_name='행정동 및 법정동코드와의 연계표')
    std.columns = std.loc[0]
    std = std[1:]
    std_seoul = std[std['시도'] == '서울특별시']
    std_df = std_seoul[['시군구','행정동\n(행정기관명)','법정동','행정기관코드','법정동코드','법정동 관할구역\n분할여부']][1:]
    std_df.columns = ['시군구','행정동','법정동','행정기관코드','법정동코드','분할여부']
    
    data = tmp_df['DONG_CODE'].value_counts()
    x = pd.DataFrame(data)
    x.rename(columns={'DONG_CODE':'HOSPITAL_NUM'},inplace=True)
    x['DONG_CODE'] = x.index
    x.index = range(len(x))

    return x

In [155]:
df = pd.read_excel('FINAL_NUM_1029.xlsx') # 법정동 데이터
copy_df = df.copy()

ps_rm_cnt = ps_remove_count()

merge = pd.merge(copy_df,ps_rm_cnt, how='outer',on='DONG_CODE')
df = merge.fillna(0)
del(df['HOSPITAL_NUM_x'])
df.rename(columns={'HOSPITAL_NUM_y':'HOSPITAL_NUM'},inplace=True)

file_name = "법정동_성형외과_제외_data.xlsx"
df.to_excel(file_name, index=False)

In [156]:
change_df = dong_change(file_name)
change_df

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,MC_NUM,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,HOSPITAL_NUM,GU,DONG,DONG_CODE
2,18.000000,0.000000,1.0,2.000000,2.000000,3.0,56.666667,2.000000,5.666667,1.000000,...,0.0,4.000000,18816.000000,7.333333,39.666667,1.666667,11.666667,종로구,청운효자동,1111051500
12,19.000000,10.000000,0.0,2.000000,0.000000,0.0,41.666667,3.000000,11.666667,5.000000,...,0.0,20.000000,22848.000000,10.333333,100.666667,2.666667,74.666667,종로구,사직동,1111053000
24,2.000000,10.000000,0.0,0.000000,1.000000,1.0,22.000000,1.000000,7.000000,1.000000,...,0.0,2.000000,14112.000000,1.000000,52.000000,1.000000,7.000000,종로구,삼청동,1111054000
31,24.000000,0.000000,1.0,4.000000,1.000000,1.0,41.000000,1.000000,5.000000,5.000000,...,0.0,8.000000,6048.000000,5.000000,30.000000,1.000000,4.000000,종로구,부암동,1111055000
34,28.000000,0.000000,0.0,0.000000,0.000000,1.0,21.000000,1.000000,7.000000,0.000000,...,0.0,6.000000,4032.000000,10.000000,36.000000,0.000000,14.000000,종로구,평창동,1111056000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,강동구,성내2동,1174065000
763,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,0.0,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,강동구,성내3동,1174066000
764,100.000000,20.000000,1.0,6.000000,1.000000,0.0,119.000000,0.000000,9.000000,10.000000,...,0.0,76.000000,4327.000000,10.000000,63.000000,1.000000,120.000000,강동구,길동,1174068500
765,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,1.000000,5.500000,2.000000,...,0.5,17.000000,2163.500000,0.000000,17.500000,1.000000,14.500000,강동구,둔촌1동,1174069000


### 버스 데이터 누락 많아서 드랍
- 사이언스 측에서 구한 데이터로 변경

In [157]:
def add_bus_data(df):
    df.drop('BUS_NUM',axis=1,inplace=True)
    copy_df = df.copy()
    copy_df['GU_DONG'] = copy_df['GU'] + copy_df['DONG']
    
    bus_df = pd.read_csv('동별_버스정보_수정.csv')
    merge_df = pd.merge(copy_df, bus_df)
    merge_df.drop('GU_DONG',axis=1,inplace=True)
    
    return merge_df

In [158]:
change_bus_df = add_bus_data(change_df)
change_bus_df

,ACADEMY_NUM,KINDER_NUM,FIRE_NUM,ELE_SCH_NUM,MID_SCH_NUM,HIGH_SCH_NUM,CCTV_NUM,POLICE_NUM,BIKE_NUM,CAR_SHR_NUM,...,CON_NUM,NOISE_VIBRATION_NUM,CHILD_MED_NUM,CAFE_NUM,PARK_NUM,HOSPITAL_NUM,GU,DONG,DONG_CODE,BUS_CNT
0,18.000000,0.000000,1.0,2.000000,2.000000,3.0,56.666667,2.000000,5.666667,1.000000,...,4.000000,18816.000000,7.333333,39.666667,1.666667,11.666667,종로구,청운효자동,1111051500,14.0
1,19.000000,10.000000,0.0,2.000000,0.000000,0.0,41.666667,3.000000,11.666667,5.000000,...,20.000000,22848.000000,10.333333,100.666667,2.666667,74.666667,종로구,사직동,1111053000,16.0
2,2.000000,10.000000,0.0,0.000000,1.000000,1.0,22.000000,1.000000,7.000000,1.000000,...,2.000000,14112.000000,1.000000,52.000000,1.000000,7.000000,종로구,삼청동,1111054000,11.0
3,24.000000,0.000000,1.0,4.000000,1.000000,1.0,41.000000,1.000000,5.000000,5.000000,...,8.000000,6048.000000,5.000000,30.000000,1.000000,4.000000,종로구,부암동,1111055000,9.0
4,28.000000,0.000000,0.0,0.000000,0.000000,1.0,21.000000,1.000000,7.000000,0.000000,...,6.000000,4032.000000,10.000000,36.000000,0.000000,14.000000,종로구,평창동,1111056000,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,강동구,성내2동,1174065000,11.0
422,63.333333,16.666667,1.0,1.333333,0.333333,0.0,52.000000,0.333333,6.000000,6.666667,...,38.666667,1442.333333,4.000000,47.666667,0.000000,59.333333,강동구,성내3동,1174066000,9.0
423,100.000000,20.000000,1.0,6.000000,1.000000,0.0,119.000000,0.000000,9.000000,10.000000,...,76.000000,4327.000000,10.000000,63.000000,1.000000,120.000000,강동구,길동,1174068500,28.0
424,38.000000,0.000000,0.0,2.000000,1.500000,1.0,32.500000,1.000000,5.500000,2.000000,...,17.000000,2163.500000,0.000000,17.500000,1.000000,14.500000,강동구,둔촌1동,1174069000,4.0
